In [ ]:
# http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture6.pdf
## Need to understand the plots. Why are the values in plot different from printed
"""
Exploding gradient problem
Solns:
added regularizations
Changed variance in initialization
Changed batch size
Decreased learning rate
"""
"""
with regularizations
lr=0.0025
batch=128
Loss: 0.556975245475769 Accuracy: 0.796875
Final test accuracy: 0.8029999732971191
"""
"""
lr = 0.0025
no regularizations
batch sizea=128
Train accuracy: 0.875
Test accuracy: 0.8687999844551086
"""
"""
lr = 0.001
batch=128
Loss: 0.7130742073059082 Accuracy: 0.7578125
Final test accuracy: 0.7803999781608582
"""

In [26]:
import tensorflow as tf
import os

from datasets import MNISTDataset
import time # first change: set up log dir and file writer(s)
logdir = os.path.join("logs", "linear" + str(time.time()))
train_writer = tf.summary.create_file_writer(os.path.join(logdir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(logdir, "test"))

# get the data
(train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.mnist.load_data()
mnist = MNISTDataset(train_imgs.reshape((-1, 784)), train_lbls,
                     test_imgs.reshape((-1, 784)), test_lbls,
                     batch_size=128, seed=int(time.time()))


# define the model first, from input to output

# this is a super deep model, cool!
n_units = 100
n_layers = 8
w_range = 0.4

# just set up a "chain" of hidden layers
layers = []
for layer in range(n_layers):
    layers.append(tf.keras.layers.Dense(
        n_units, activation=tf.nn.relu,
        kernel_initializer=tf.initializers.RandomUniform(minval=-w_range,
                                                         maxval=w_range),
        bias_initializer=tf.initializers.constant(0.001),
        activity_regularizer=tf.keras.regularizers.l2(0.1)))

# finally add the output layer
layers.append(tf.keras.layers.Dense(
    10, kernel_initializer=tf.initializers.RandomUniform(minval=-w_range,
                                                         maxval=w_range),
        bias_initializer=tf.initializers.constant(0.001),
        activity_regularizer=tf.keras.regularizers.l2(0.1)))
"""

        kernel_regularizer=tf.keras.regularizers.l1(0.3),
        bias_regularizer=tf.keras.regularizers.l1(0.3),

# just set up a "chain" of hidden layers
layers = []
for layer in range(n_layers):
    layers.append(tf.keras.layers.Dense(
        n_units, activation=tf.nn.relu,
        kernel_initializer=tf.initializers.RandomUniform(minval=-w_range,
                                                         maxval=w_range)))

# finally add the output layer
layers.append(tf.keras.layers.Dense(
    10, kernel_initializer=tf.initializers.RandomUniform(minval=-w_range,
                                                         maxval=w_range),
        bias_initializer=tf.initializers.constant(0.001)))
"""
lr = 0.0025
for step in range(2000):
    img_batch, lbl_batch = mnist.next_batch()
    with tf.GradientTape() as tape:
        # here we just run all the layers in sequence via a for-loop
        out = img_batch
        for layer in layers:
            out = layer(out)
        xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=out, labels=lbl_batch))

    weights = [var for l in layers for var in l.trainable_variables]
    grads = tape.gradient(xent, weights)
    for grad, var in zip(grads, weights):
        var.assign_sub(lr*grad)
    i=0        
    with train_writer.as_default():
        tf.summary.scalar("loss", xent, step=step)
        tf.summary.histogram("logits", out, step=step)
        for W in weights:
            tf.summary.histogram("weights"+str(i), W, step=step)
            i+=1
    tf.summary.scalar("Loss", xent, step=step)
    if not step % 100:
        preds = tf.argmax(out, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch), tf.float32))
        print("Loss: {} Accuracy: {}".format(xent, acc))
        with train_writer.as_default():
            tf.summary.scalar("Trainaccuracy", acc, step=step)
            tf.summary.image("input", tf.reshape(img_batch, [-1, 28, 28, 1]), step=step)


out = mnist.test_data
for layer in layers:
    out = layer(out)
test_preds = tf.argmax(out, axis=1, output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, mnist.test_labels), tf.float32))
with test_writer.as_default():
    tf.summary.scalar("Testaccuracy", acc, step=step)
print("Final test accuracy: {}".format(acc))


Loss: 104.44222259521484 Accuracy: 0.0546875
Loss: 2.529419422149658 Accuracy: 0.4609375
Loss: 1.940325140953064 Accuracy: 0.4140625
Loss: 1.449946641921997 Accuracy: 0.5234375
Loss: 1.214517593383789 Accuracy: 0.640625
Starting new epoch...
Loss: 1.021724820137024 Accuracy: 0.6875
Loss: 0.9539246559143066 Accuracy: 0.6953125
Loss: 0.7553335428237915 Accuracy: 0.7109375
Loss: 0.7975450754165649 Accuracy: 0.703125
Loss: 0.7668576240539551 Accuracy: 0.703125
Starting new epoch...
Loss: 0.5632084608078003 Accuracy: 0.8046875
Loss: 0.7695791721343994 Accuracy: 0.78125
Loss: 0.863564133644104 Accuracy: 0.71875
Loss: 0.8433808088302612 Accuracy: 0.6484375
Loss: 0.7873533964157104 Accuracy: 0.7421875
Starting new epoch...
Loss: 0.8399887084960938 Accuracy: 0.734375
Loss: 0.6575472354888916 Accuracy: 0.8203125
Loss: 0.6086424589157104 Accuracy: 0.7890625
Loss: 0.5494330525398254 Accuracy: 0.8515625
Starting new epoch...
Loss: 0.5667954683303833 Accuracy: 0.8046875
Final test accuracy: 0.817900

In [36]:
%load_ext tensorboard

%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 20316), started 1 day, 1:13:23 ago. (Use '!kill 20316' to kill it.)

In [34]:
os.path.join(os.getcwd(),'logs')


'C:\\Users\\user\\Desktop\\Ovgu Educational\\IDL\\Ass2\\EPIC_DEEP_LEARNING_FAIL_COMPILATION_OCTOBER_2019\\logs'